# BioExplorer - Metabolism - Combine movies into gallery view
![](../bioexplorer_metabolism_banner.png)

In [ ]:
from glob import glob
import os
movies_folder = os.getenv('METABOLISM_MOVIE_FOLDER')

## Media production

### Create movies from frames

In [ ]:
all_files = glob(os.path.join(movies_folder, '*'))
all_folders = list()
for file in all_files:
    if os.path.isdir(file):
        all_folders.append(file)
for folder in all_folders:
    basename = os.path.basename(folder)
    movie_filename = os.path.join(movies_folder, basename + '_1k_v13.mp4')
    cmd = 'ffmpeg -f Image2 -i %s/%%*.png -b:v 16M -threads 16 -y %s 2>/dev/null' % (folder, movie_filename)
    print(basename)
    os.system(cmd)

### Combine movies into mosaic

In [ ]:
all_movie_files = glob(os.path.join(movies_folder, '*.mp4'))
movie_files=all_movie_files
print('Found %d movies' % len(movie_files))

In [ ]:
nb_columns = 5
nb_lines = 5
resolution=[3840, 2160]
image_size=[int(3840 / nb_columns), int(2160 / nb_lines)]

print(str(nb_columns) + 'x' + str(nb_lines) + '=' + str(nb_lines*nb_columns))
print(image_size)

In [ ]:

def generate_mosaic(files, output_filename):
    cmd = 'ffmpeg'
    for movie_file in files:
        cmd = cmd + ' -i ' + movie_file
    cmd = cmd + ' -filter_complex "nullsrc=size=' + str(resolution[0]) + 'x' + str(resolution[1]) +' [base];'

    nb_movies = len(files)

    for i in range(nb_movies):
        cmd = cmd + ' [%d:v] setpts=PTS-STARTPTS, scale=%dx%d [vid%d];' % (i, image_size[0], image_size[1], i)

    for i in range(nb_movies):
        if i==0:
            cmd = cmd + ' [base]'
        else:
            cmd = cmd + ' [tmp%d]' % (i-1)
        cmd = cmd + '[vid%d] overlay=shortest=1:x=%d:y=%d' % (i, int(i % nb_columns) * image_size[0], (int(i / nb_columns) * image_size[1]))
        if i!=nb_movies-1:
            cmd = cmd + ' [tmp%d];' % i;
    cmd = cmd + '" -c:v libx264 -y ' + output_filename + ' 2>/dev/null'
    # print(cmd)
    os.system(cmd)

concentrations_output_filename = os.path.join(movies_folder, 'metabolites_concentrations_4k_v13.mp4')
background_output_filename = os.path.join(movies_folder, 'metabolites_background_4k_v13.mp4')

if os.path.exists(concentrations_output_filename):
    print('Removing ' + concentrations_output_filename)
    os.remove(concentrations_output_filename)
if os.path.exists(background_output_filename):
    print('Removing ' + background_output_filename)
    os.remove(background_output_filename)

all_movie_files = glob(os.path.join(movies_folder, '*.mp4'))
movie_files=all_movie_files
print('Found %d movies' % len(movie_files))
generate_mosaic(movie_files, concentrations_output_filename)

image_files = list()
for i in range(len(movie_files)):
    image_files.append(os.path.join(movies_folder, 'mosaic', 'metabolism_bioexplorer_2k_v13.png'))
generate_mosaic(image_files, background_output_filename)


### Extract frames from mosaic movie

In [ ]:
from PIL import Image
src_image = Image.open(os.path.join(movies_folder, 'mosaic', 'metabolism_bioexplorer_2k_v13.png'))

src_image.thumbnail(image_size)
dst_image = new_im = Image.new('RGBA', (3840,2160))

for i in range(nb_columns):
    for j in range(nb_lines):
        src_image=Image.eval(src_image,lambda x: x+(i+j)/30)
        dst_image.paste(src_image, (i * image_size[0],j * image_size[1]))

dst_image.save(os.path.join(movies_folder, 'mosaic', 'metabolism_gallery_4k_v13.png'))

In [ ]:
concentrations_output_frames = os.path.join(movies_folder, 'mosaic', 'concentrations', '%05d.png')
cmd = 'ffmpeg -i ' + concentrations_output_filename + ' ' + concentrations_output_frames + ' 2>/dev/null'
os.system(cmd)